# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2025-04-10 ~ 2025-04-17
* 코드 작성 목적: NLP

# 1. 기초 설정

## 사용 라이브러리

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from konlpy.tag import *
from collections import Counter

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 처리

## A. 크롤링 데이터 불러오기

In [4]:
crawled = pd.read_csv("../data/dbpia_crawling_result.csv")

In [5]:
print(crawled.shape)

(800, 3)


In [6]:
print(crawled.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   카테고리    800 non-null    object
 1   제목      800 non-null    object
 2   초록      800 non-null    object
dtypes: object(3)
memory usage: 18.9+ KB
None


In [7]:
crawled[crawled['초록'].duplicated(keep=False)]

,카테고리,제목,초록
3,인문학,한국의 초저출산: 무엇이 원인이고 무엇이 해법인가,등록된 정보가 없습니다.
5,인문학,"왜 한국 남성은 한국여성들에게 분노하는가 : 여성혐오, 한국사회가 가지고 있는 어떤...",등록된 정보가 없습니다.
12,인문학,강남역 살인사건부터 '메갈리아' 논쟁까지 : '페미니즘 봉기'와 한국 남성성의 위기,등록된 정보가 없습니다.
16,인문학,"방탄소년단: 새로운 세대의 새로운 소통 방식, 그리고 감정노동",등록된 정보가 없습니다.
17,인문학,GMO(유전자 조작 식품)를 어떻게 볼 것인가?,등록된 정보가 없습니다.
...,...,...,...
768,복합학,"감정, 뇌과학으로 바라보다",등록된 정보가 없습니다.
772,복합학,코로나 19가 불러온 변화들,등록된 정보가 없습니다.
780,복합학,위드 코로나 시대의 라이프스타일 변화와 新소비 트렌드 - MZ 세대를 중심으로,등록된 정보가 없습니다.
788,복합학,메타버스를 활용한 역사교육콘텐츠 개발 방안,등록된 정보가 없습니다.


In [8]:
crawled[crawled['초록']=='등록된 정보가 없습니다.']

,카테고리,제목,초록
3,인문학,한국의 초저출산: 무엇이 원인이고 무엇이 해법인가,등록된 정보가 없습니다.
5,인문학,"왜 한국 남성은 한국여성들에게 분노하는가 : 여성혐오, 한국사회가 가지고 있는 어떤...",등록된 정보가 없습니다.
12,인문학,강남역 살인사건부터 '메갈리아' 논쟁까지 : '페미니즘 봉기'와 한국 남성성의 위기,등록된 정보가 없습니다.
16,인문학,"방탄소년단: 새로운 세대의 새로운 소통 방식, 그리고 감정노동",등록된 정보가 없습니다.
17,인문학,GMO(유전자 조작 식품)를 어떻게 볼 것인가?,등록된 정보가 없습니다.
...,...,...,...
768,복합학,"감정, 뇌과학으로 바라보다",등록된 정보가 없습니다.
772,복합학,코로나 19가 불러온 변화들,등록된 정보가 없습니다.
780,복합학,위드 코로나 시대의 라이프스타일 변화와 新소비 트렌드 - MZ 세대를 중심으로,등록된 정보가 없습니다.
788,복합학,메타버스를 활용한 역사교육콘텐츠 개발 방안,등록된 정보가 없습니다.


In [9]:
# '등록된 정보가 없습니다.'를 빈 문자열로 대체
crawled['초록'] = crawled['초록'].replace('등록된 정보가 없습니다.', '')
crawled[crawled.duplicated(subset=['제목','초록'], keep=False)]

,카테고리,제목,초록
106,사회과학,유튜브 알고리즘과 확증편향,"알고리즘을 이용한 동영상 추천 시스템을 통해 넷플릭스, 유튜브 등 다양한 OTT 서..."
306,공학,유튜브 알고리즘과 확증편향,"알고리즘을 이용한 동영상 추천 시스템을 통해 넷플릭스, 유튜브 등 다양한 OTT 서..."
719,복합학,"감정, 뇌과학으로 바라보다",
768,복합학,"감정, 뇌과학으로 바라보다",


In [10]:
# 제목 + 초록 기준 중복 제거
crawled = crawled.drop_duplicates(subset=['제목', '초록'], keep='first').reset_index(drop=True)

In [11]:
# 제목과 초록을 합쳐 문서 생성
docs = [f"{title} {abstract}" for title, abstract in zip(crawled['제목'], crawled['초록'])]

In [12]:
docs[:10]

['썸타기와 어장관리에 대한 철학적 고찰 최근 한국 사회에서 유통되기 시작한 신조어 ‘썸탄다’는 흔히 본격적인 연인관계로 발전하기 이전 단계의 애매한 남녀관계를 뜻하는 것으로 이해된다. 그러나 그것이 정확히 어떤 의미를 지니는지, 썸탄다는 활동의 정체가 정확히 무엇인지는 여전이 오리무중이다. 열 사람에게 물어보면 열 사람 모두 다른 답변을 내놓을 지경이다. 이런 배경 하에서 필자는 미국의 철학자 해리 프랑크푸르트(Harry Frankfurt)의 인간관에 의거하여 의지적 불확정성 개념을 도입하고, 그 개념을 통해서 썸타기의 본성을 포착할 것이다. 필자의 제안은 상대방에 대한 이성적 호감을 지닌 두 남녀가 새롭게 만남을 시작하며 자신들의 의지적 불확정성에 대하여 서로 영향을 주고 받는 과정이 썸타기의 핵심이라는 것이다. 썸타기에 대한 필자의 이러한 제안은 썸타기와 어장관리의 관계에 대한 흥미로운 시각을 제공해 줄 것인데, 그에 대해서는 본 논문의 말미에서 상세히 논의될 것이다.',
 '인공 지능에서 인공 감정으로 : 감정을 가진 기계는 실현가능한가? 인공 감정에 관한 철학적 탐구가 필요한 시점이다. 인간의 고유한 영역으로 간주되던 인지적 과제에서 기계의 추월을 염려하는 처지가 되자, 사람들은 이제 이성이 아니라 감정에서 인간의 고유성을 찾으려한다. 하지만 최근 인공지능 로봇에 감성을 불어넣는 작업이 새로운 화두로 떠오르고 있다. 이 글은 인공 감정의 실현가능성과 잠재적인 위험을 논의한다. 먼저, 감성 로봇의 개발 현황과 주요한 동기들을 개괄하고, 왜 로봇의 감정이 문제가 되는지 살펴본다. 진정한 감정-소유 로봇이 가능한지 검토하기 위해 감정을 선험적으로 정의하기보다는 감정이 수행하는 몇 가지 핵심 역할을 소개하고 이로부터 어떤 대상에 감정을 부여할 수 있는 기준들을 제안한다. 나는 이런 기준에 비추어 진정한 감정 로봇이 근미래에 실현될 가능성이 낮다고 주장한다. 그러나 감정-소유 로봇이 등장하기 이전이라도, 어느 정도의 자율성을 가진 로봇과 맺는 일방적 감정 소통

## B. 구두점 처리

In [13]:
# 제거할 문자와 그 대체값을 딕셔너리로 정의
remove_dict = {
    '.': '',   # 마침표 제거
    ',': '',   # 쉼표 제거
    "'": '',   # 작은따옴표 제거
    '"': '',   # 큰따옴표 제거
    '·': ' ',  # 중간 점을 공백으로 대체
    '=': ' ',   # 등호 제거
    '\\n': ' ',  # 개행 문자 제거
    ':': ' ',
    '[': '',
    ']': '',
    '…': '',
    '\\': '',
    '‘': '',
    '’': ''
}

In [14]:
# 여러 문자 제거를 위한 함수 생성
def clean_text(text, remove_dict):
    for key, value in remove_dict.items():
        text = text.replace(key, value)
    return text

In [15]:
cleaned_docs = [clean_text(doc, remove_dict) for doc in docs]

In [16]:
cleaned_docs[:10]

['썸타기와 어장관리에 대한 철학적 고찰 최근 한국 사회에서 유통되기 시작한 신조어 썸탄다는 흔히 본격적인 연인관계로 발전하기 이전 단계의 애매한 남녀관계를 뜻하는 것으로 이해된다 그러나 그것이 정확히 어떤 의미를 지니는지 썸탄다는 활동의 정체가 정확히 무엇인지는 여전이 오리무중이다 열 사람에게 물어보면 열 사람 모두 다른 답변을 내놓을 지경이다 이런 배경 하에서 필자는 미국의 철학자 해리 프랑크푸르트(Harry Frankfurt)의 인간관에 의거하여 의지적 불확정성 개념을 도입하고 그 개념을 통해서 썸타기의 본성을 포착할 것이다 필자의 제안은 상대방에 대한 이성적 호감을 지닌 두 남녀가 새롭게 만남을 시작하며 자신들의 의지적 불확정성에 대하여 서로 영향을 주고 받는 과정이 썸타기의 핵심이라는 것이다 썸타기에 대한 필자의 이러한 제안은 썸타기와 어장관리의 관계에 대한 흥미로운 시각을 제공해 줄 것인데 그에 대해서는 본 논문의 말미에서 상세히 논의될 것이다',
 '인공 지능에서 인공 감정으로   감정을 가진 기계는 실현가능한가? 인공 감정에 관한 철학적 탐구가 필요한 시점이다 인간의 고유한 영역으로 간주되던 인지적 과제에서 기계의 추월을 염려하는 처지가 되자 사람들은 이제 이성이 아니라 감정에서 인간의 고유성을 찾으려한다 하지만 최근 인공지능 로봇에 감성을 불어넣는 작업이 새로운 화두로 떠오르고 있다 이 글은 인공 감정의 실현가능성과 잠재적인 위험을 논의한다 먼저 감성 로봇의 개발 현황과 주요한 동기들을 개괄하고 왜 로봇의 감정이 문제가 되는지 살펴본다 진정한 감정-소유 로봇이 가능한지 검토하기 위해 감정을 선험적으로 정의하기보다는 감정이 수행하는 몇 가지 핵심 역할을 소개하고 이로부터 어떤 대상에 감정을 부여할 수 있는 기준들을 제안한다 나는 이런 기준에 비추어 진정한 감정 로봇이 근미래에 실현될 가능성이 낮다고 주장한다 그러나 감정-소유 로봇이 등장하기 이전이라도 어느 정도의 자율성을 가진 로봇과 맺는 일방적 감정 소통은 잠재적으로 위험할 수 있으며 이에 대

## C. 불용어 및 자연어 처리
---
- 표준 불용어 목록은 아래 논문에서 제시한 시안 사용
    - [DBpia - 텍스트마이닝을 위한 한국어 불용어 목록 연구](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE07540294)
        - 길호현. (2018). 텍스트마이닝을 위한 한국어 불용어 목록 연구. 우리말글, 78, 1-25.
- 형태소 분석기는 아래를 참고하여 선정
    - [DBpia - 형태소 분석기의 품사별 정확성 분석](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11082554)
        - 김수연, 안석호, 김동현, 이의종, 서영덕. (2021-11-25). 형태소 분석기의 품사별 정확성 분석. Proceedings of KIIT Conference, 제주.
        - 정확성 비교
    - [KoNLPy - 형태소 분석 및 품사 태깅](https://konlpy.org/ko/latest/morph/#pos-tagging-with-konlpy)
        - 실행 시간 및 정확성 비교
    - [blog - 한국어 형태소분석기를 비교하는 기준](https://jessymin.github.io/text%20analysis/2022/08/21/text-analysis-korean-stem-analyzers.html)
        - 사용자 사전 존재 인지
    - [blog - [NLP] 코모란(Komoran) 사용자사전 만들기](https://needjarvis.tistory.com/741)
        - 사용자 사전 존재 확인
    - komoran
        - (김수연 외, 2022)에서 가장 뛰어난 성능을 보여줌
        - 체언, 특히 합성명사에 강하며 논문 제목은 대개 체언으로 이루어짐을 고려
        - 어미에 강하여 초록에서도 사용하기에 적합하다고 판단함
        - 사용자 사전을 사용할 수 있음
    - okt
        - 구 트위터에서 출발한 만큼 신조어에 강함

In [17]:
# 불용어 리스트 파일 경로
stopword_file = '../data/korean_stopwords.txt'

# 불용어 리스트 불러오기
with open(stopword_file, 'r', encoding='utf-8') as f:
    default_stopwords = [line.strip() for line in f if line.strip()]

In [18]:
default_stopwords[:10]

['가량', '가지', '각', '간', '갖은', '개', '개국', '개년', '개소', '개월']

In [19]:
print(len(default_stopwords))

293


In [20]:
# 형태소 분석기 정의
okt = Okt()
komoran = Komoran()

In [21]:
## 단어 등장 빈도 추출 -> 도메인 특이 불용어 확인
def print_topN_nouns(tagger, top_n=30):
    # 전체 문서에서 명사 추출
    all_nouns = []
    for doc in cleaned_docs:
        nouns = tagger.nouns(doc) # 형태소 분석을 통해 명사 추출
        # 불용어 제거 및 길이가 1 이상인 명사만 사용
        filtered = [noun for noun in nouns if noun not in default_stopwords and len(noun) > 1]
        all_nouns.extend(filtered)

    # 단어 빈도 계산
    noun_freq = Counter(all_nouns)

    # 상위 N개 추출
    most_common_nouns = noun_freq.most_common(top_n)

    # 출력
    print(f"[상위 {top_n}개 단어]")
    for word, freq in most_common_nouns:
        print(f"{word}: {freq}")

In [22]:
print_topN_nouns(komoran, 30)

[상위 30개 단어]
연구: 1153
분석: 516
사회: 475
결과: 418
기술: 363
영향: 344
변화: 336
필요: 278
교육: 275
사용: 264
인간: 260
문제: 256
활용: 246
효과: 231
환경: 225
산업: 220
중심: 215
인공지능: 214
개발: 210
관련: 207
방법: 200
기능: 199
제시: 198
디자인: 193
서비스: 192
조사: 191
이용: 189
증가: 182
식품: 181
스트레스: 181


In [23]:
print_topN_nouns(okt, 30)

[상위 30개 단어]
연구: 1037
대한: 660
분석: 506
사회: 470
결과: 399
기술: 372
통해: 346
영향: 346
변화: 312
사용: 266
교육: 262
위해: 261
인간: 259
인공: 252
이용: 251
지능: 250
활용: 248
문제: 241
효과: 235
환경: 216
중심: 216
개발: 212
관련: 209
기능: 205
디자인: 201
방법: 200
서비스: 193
대해: 191
문화: 191
스트레스: 187


In [24]:
# 사용자 정의 불용어 리스트
custom_stopwords = [
    '연구', '대한', '분석', '결과', '통해',
    '영향', '위해', '문제', '효과', '방법'
]

In [25]:
# 불용어 리스트 결합
combined_stopwords = default_stopwords + custom_stopwords

print(combined_stopwords[-15:-5])

['현', '호', '혹은', '회', '흥', '연구', '대한', '분석', '결과', '통해']


In [26]:
terms_komoran = []

for w in cleaned_docs:
  tokens = komoran.nouns(w)
  stop_tokens = [i for i in tokens if not i in combined_stopwords]
  terms_komoran.append(stop_tokens)

In [27]:
print(terms_komoran[:5])

[['어장', '관리', '철학', '고찰', '최근', '한국', '사회', '유통', '시작', '신조어', '연인', '관계', '발전', '이전', '단계', '남녀', '관계', '뜻', '이해', '의미', '활동', '정체', '여전', '오리무중', '사람', '사람', '답변', '배경', '하', '필자', '미국', '철학자', '해리', '프랑크푸르트', '인간관', '의거', '의지', '불확정성', '개념', '도입', '개념', '본성', '포착', '필자', '제안', '상대방', '성적', '호감', '남녀', '만남', '시작', '자신', '의지', '불확정성', '과정', '핵심', '필자', '제안', '어장', '관리', '관계', '흥미', '시각', '제공', '논문', '말미', '논의'], ['인공', '지능', '인공', '감정', '감정', '기계', '실현', '인공', '감정', '철학', '탐구', '필요', '시점', '인간', '고유', '영역', '간주', '인지', '과제', '기계', '추월', '염려', '처지', '사람', '이성', '감정', '인간', '고유', '최근', '인공지능', '로봇', '감성', '작업', '화두', '글', '인공', '감정', '실현', '잠재', '위험', '논의', '감성', '로봇', '개발', '현황', '동기', '개괄', '로봇', '감정', '감정', '소유', '로봇', '검토', '감정', '선험', '정의', '감정', '수행', '핵심', '역할', '소개', '대상', '감정', '부여', '기준', '제안', '기준', '감정', '로봇', '미래', '실현', '주장', '감정', '소유', '로봇', '등장', '이전', '정도', '자율', '로봇', '일방', '감정', '소통', '잠재', '위험', '대비', '주장'], ['논문', '철회', '관음', '충', '발생학', '한국', '남성성', '완전', '변태', '과

In [28]:
terms_okt = []

for w in cleaned_docs:
  tokens = okt.nouns(w)
  stop_tokens = [i for i in tokens if not i in combined_stopwords]
  terms_okt.append(stop_tokens)

In [29]:
print(terms_okt[:5])

[['썸', '어장', '관리', '철학', '고찰', '최근', '한국', '사회', '유통', '시작', '신조어', '썸', '본격', '연인', '관계', '발전', '이전', '단계', '남녀', '관계', '뜻', '이해', '의미', '썸', '활동', '정체', '여전', '오리무중', '열', '사람', '열', '사람', '모두', '답변', '배경', '하', '필자', '미국', '철학자', '해리', '프랑크푸르트', '의', '인간', '관', '의거', '의지', '불확정성', '개념', '도입', '개념', '썸', '본성', '포착', '필자', '제안', '상대방', '성적', '호감', '남녀', '만남', '시작', '자신', '의지', '불확정성', '대하', '서로', '과정', '썸', '핵심', '썸', '필자', '제안', '썸', '어장', '관리', '관계', '시각', '제공', '대해', '논문', '말미', '상세', '논의'], ['인공', '지능', '인공', '감정', '감정', '기계', '실현', '인공', '감정', '관', '철학', '탐구', '시점', '인간', '고유', '영역', '간주', '인지', '과제', '기계', '추월', '염려', '처지', '사람', '이제', '이성', '감정', '인간', '유성', '최근', '인공', '지능', '로봇', '감성', '불어', '작업', '화두', '글', '인공', '감정', '실현', '가능성', '잠재', '위험', '논의', '먼저', '감성', '로봇', '개발', '현황', '주요한', '동기', '개괄', '왜', '로봇', '감정', '감정', '소유', '로봇', '검토', '감정', '선', '정의', '감정', '수행', '핵심', '역할', '소개', '이로', '대상', '감정', '부여', '기준', '제안', '기준', '감정', '로봇', '미래', '실현', '가능성', '주장', '감정', '소유', '

# 3. 결과 저장

## 각 문서를 하나의 문자열로 합치기

In [30]:
final_cleaned_docs_komoran = [' '.join(doc) for doc in terms_komoran]
print('komoran:')
print(final_cleaned_docs_komoran[:5])

komoran:
['어장 관리 철학 고찰 최근 한국 사회 유통 시작 신조어 연인 관계 발전 이전 단계 남녀 관계 뜻 이해 의미 활동 정체 여전 오리무중 사람 사람 답변 배경 하 필자 미국 철학자 해리 프랑크푸르트 인간관 의거 의지 불확정성 개념 도입 개념 본성 포착 필자 제안 상대방 성적 호감 남녀 만남 시작 자신 의지 불확정성 과정 핵심 필자 제안 어장 관리 관계 흥미 시각 제공 논문 말미 논의', '인공 지능 인공 감정 감정 기계 실현 인공 감정 철학 탐구 필요 시점 인간 고유 영역 간주 인지 과제 기계 추월 염려 처지 사람 이성 감정 인간 고유 최근 인공지능 로봇 감성 작업 화두 글 인공 감정 실현 잠재 위험 논의 감성 로봇 개발 현황 동기 개괄 로봇 감정 감정 소유 로봇 검토 감정 선험 정의 감정 수행 핵심 역할 소개 대상 감정 부여 기준 제안 기준 감정 로봇 미래 실현 주장 감정 소유 로봇 등장 이전 정도 자율 로봇 일방 감정 소통 잠재 위험 대비 주장', '논문 철회 관음 충 발생학 한국 남성성 완전 변태 과정 추이 물질주의 논고 대한민국 사회 관통 불법 촬영 물 특정 포획 물 기반 분포 관음 충 형태 발생학 고찰 형태 발생학 고찰 대한민국 사회 문화 환경 안 디지털 성범죄 시스템 추동 관음 충 특정 군집 구성체 젠더 조건 중심 발생 생장 증식 추적 의미 필자 남 유충 관음 충 한남 충 용어 배태 곤충 집체 형태 발생학 착상 적극 활용 논의 배경 남 충 알 유충 성충 단계 탈피 성장 형태 상태 유지 완전 변태 모델 남 유충 남 충 변태 과정 추이 관음 충 지수 구체 방식 단계 진화 필자 미국 물질주의 이론 선두 주자 이자 들뢰즈 현대 계승자 마뉴 엘 이론 틀 창의 해석 관음 충 집체 발생 진화 과정 잠재 궤적 방향 현실 양태 비판 시도 제시 군집 구성체 사고 틀 한국 남아 관음 충 집단 생장 진화 사회 문화 자가 복제 기제 강렬도 사고 틀 관음 충 형태 발생학 변화 과정 영토 코드 기준 척도 각각 변이 국면 특이점 파악 고찰 위상 학 사고 틀 가부 

In [31]:
final_cleaned_docs_okt = [' '.join(doc) for doc in terms_okt]
print('okt:')
print(final_cleaned_docs_okt[:5])

okt:
['썸 어장 관리 철학 고찰 최근 한국 사회 유통 시작 신조어 썸 본격 연인 관계 발전 이전 단계 남녀 관계 뜻 이해 의미 썸 활동 정체 여전 오리무중 열 사람 열 사람 모두 답변 배경 하 필자 미국 철학자 해리 프랑크푸르트 의 인간 관 의거 의지 불확정성 개념 도입 개념 썸 본성 포착 필자 제안 상대방 성적 호감 남녀 만남 시작 자신 의지 불확정성 대하 서로 과정 썸 핵심 썸 필자 제안 썸 어장 관리 관계 시각 제공 대해 논문 말미 상세 논의', '인공 지능 인공 감정 감정 기계 실현 인공 감정 관 철학 탐구 시점 인간 고유 영역 간주 인지 과제 기계 추월 염려 처지 사람 이제 이성 감정 인간 유성 최근 인공 지능 로봇 감성 불어 작업 화두 글 인공 감정 실현 가능성 잠재 위험 논의 먼저 감성 로봇 개발 현황 주요한 동기 개괄 왜 로봇 감정 감정 소유 로봇 검토 감정 선 정의 감정 수행 핵심 역할 소개 이로 대상 감정 부여 기준 제안 기준 감정 로봇 미래 실현 가능성 주장 감정 소유 로봇 등장 이전 정도 자율성 로봇 일방 감정 소통 잠재 대비 주장', '논문 철회 관음 충의 발생학 한국 남성성 불완전 변태 과정 의 추이 물질 주의 논고 대한민국 사회 관통 불법 촬영 물이 특정 포획물 기반 분포 관음 충 형태 발생학 고찰 형태 발생학 고찰 대한민국 사회 문화 환경 안 디지털 성범죄 시스템 추 키 관음 충 특정 군집 성체 젠더 조건 중심 발생 생장 증식 거듭 추적 함 의미 필자 한남 유충 관음 충 남충 용어 배태 곤충 군집 형태 발생학 착상 적극 활용 논의 배경 삼고 남충 알 유충 성충 단계 탈피 성장 거듭 형태 상태 유지 불완전 변태 의 모델 한남 유충 남충 변태 과정 추이 관음 충의 지수 를 구체 방식 단계 진화 필자 미국 물질 주의 이론 선두 주자 이자 들뢰즈 현대 계승 마뉴 데란 의 이론 틀 창의 재해 함 관음 충 군집 발생 진화 과정 잠재 궤 방향 현실 양태 비판 시도 첫째 데란 다가 제시 군집 성체 사고 틀 한국 남아 관음 충 집단 생장 진화

## 결과를 DataFrame으로 변환

In [32]:
df_cleaned_komoran = pd.DataFrame({'cleaned_doc': final_cleaned_docs_komoran})
df_cleaned_komoran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cleaned_doc  798 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [33]:
df_cleaned_okt = pd.DataFrame({'cleaned_doc': final_cleaned_docs_okt})
df_cleaned_okt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cleaned_doc  798 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


## CSV 파일로 저장

In [34]:
df_cleaned_komoran.to_csv('../data/cleaned_docs_komoran.csv', index=False, encoding='utf-8-sig')
df_cleaned_okt.to_csv('../data/cleaned_docs_okt.csv', index=False, encoding='utf-8-sig')


## 잘 저장되었는지 확인

In [35]:
test_komoran = pd.read_csv('../data/cleaned_docs_komoran.csv')
test_komoran.head(1)

,cleaned_doc
0,어장 관리 철학 고찰 최근 한국 사회 유통 시작 신조어 연인 관계 발전 이전 단계 ...


In [36]:
test_okt = pd.read_csv('../data/cleaned_docs_okt.csv')
test_okt.head(1)

,cleaned_doc
0,썸 어장 관리 철학 고찰 최근 한국 사회 유통 시작 신조어 썸 본격 연인 관계 발전...
